# HetioNet based Metapath Extraction

In this notebook we will showcase the efficiency of PAM on Metapath Extraction.

We will use the [HetioNet](https://github.com/hetio/hetionet) KG for this task.

The process is simple:
1. First download the dataset. The .tsv with the triples is expected to be in folder in the same directory as this notebook with the name "data".
2. Create the PAMs P and P^2 from the collection of triples.
3. Find metapaths that may imply the treats relation based on a simple check: $P[C,D] = ``Compound$  $treats$ $Disease"$ and $P^2[C,D] = v \neq 0$.

According to 3, the metapath $m_1$ that we will extract in that case is will be: $m_1: v \implies "treats"$.

Finally, after aggregating all Metapaths, we will check for scenarios were the metapath can maybe provide us with new possible treatmens with an explanation in place.

## Initial imports

In [2]:
%load_ext autoreload
%autoreload 2

import scipy
import tqdm
import time
import sparse

import numpy as np
import pandas as pd

from itertools import permutations, product
from scipy.sparse import csr_matrix
from collections import defaultdict


from utils import (
    get_prime_map_from_rel,
    get_primefactors,
    load_data,
    get_sparsity
)

In [3]:
import os
import random


def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
#   torch.manual_seed(seed)
#   torch.cuda.manual_seed(seed)
#   torch.backends.cudnn.deterministic = True
#   os.environ('PYTHONHASHSEED') = str(seed)
set_all_seeds(42)

# Loading the data

We load the data. Because in the original work they allow path traverals that do not follow the original direction, we also add the inverse edges between nodes to allow for such paths.

In [72]:
time_s = time.time()
add_inverse_edges = "YES"
df_train = pd.read_csv("./data/Hetionet/hetionet-v1.0-edges.tsv", sep='\t')
df_train.columns = ['head', 'rel', 'tail']
df_train.dropna(inplace=True)
print(f'Original # of unique rels: {df_train["rel"].unique()}')

if "YES" in add_inverse_edges:
    print(f"Will add the inverse train edges as well..")
    df_train["rel"] = df_train["rel"].astype(str)
    df_train_inv = df_train.copy()
    df_train["rel"] = df_train["rel"].apply(lambda x: x[::-1])
    df_train_inv["head"] = df_train["tail"]
    df_train_inv["tail"] = df_train["head"]
    if add_inverse_edges == "YES__INV":
        df_train_inv["rel"] = df_train["rel"] + "__INV"
    df_train = df_train.append(df_train_inv)
    print(f'Before deduplication: {df_train.shape[0]}')
    df_train.drop_duplicates(inplace=True)  
    print(f'After deduplication: {df_train.shape[0]}')

unique_rels = sorted(list(df_train['rel'].unique()))
unique_nodes = sorted(set(df_train['head'].values.tolist() + df_train['tail'].values.tolist()))
print(f'# of unique rels: {len(unique_rels)} \t | # of unique nodes: {len(unique_nodes)}')


time_prev = time.time()
time_needed = time_prev - time_s

Original # of unique rels: ['GpBP' 'GiG' 'CrC' 'DdG' 'DpS' 'DlA' 'CtD' 'CbG' 'CuG' 'DrD' 'DaG' 'CpD'
 'AdG' 'AuG' 'GcG' 'GpMF' 'PCiC' 'GpCC' 'Gr>G' 'CdG' 'DuG' 'GpPW' 'CcSE'
 'AeG']
Will add the inverse train edges as well..


/tmp/ipykernel_3606630/646125673.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train = df_train.append(df_train_inv)


Before deduplication: 4500394
After deduplication: 4500394
# of unique rels: 44 	 | # of unique nodes: 45158


# Map the relations and nodes

The relationsa are mapepd to primes according to PAM paradigm and nodes to integers for the rows, columns of the matrix.

In [73]:

time_s = time.time()

node2id = {}
id2node = {}
for i, node in enumerate(unique_nodes):
    node2id[node] = i
    id2node[i] = node



rel2id, id2rel = get_prime_map_from_rel(unique_rels, 
                                        starting_value=2, 
                                        spacing_strategy="step_100")


time_prev = time.time()
time_needed = time_prev - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')

Total time: 0.01345 secs (0.00 mins)


In [74]:
original_rel2id = {}
for i, (key, value) in enumerate(rel2id.items()):
    if i == len(rel2id) / 2:
        break
    else:
        original_rel2id[key] = value
print(original_rel2id)
if add_inverse_edges == 'YES__INV':
    assert len(original_rel2id) * 2 == len(rel2id)
rel2id

{'AdG': 3, 'AeG': 107, 'AlD': 211, 'AuG': 313, 'CCpG': 419, 'CbG': 521, 'CcSE': 631, 'CdG': 733, 'CiCP': 839, 'CpD': 941, 'CrC': 1049, 'CtD': 1151, 'CuG': 1259, 'DaG': 1361, 'DdG': 1471, 'DlA': 1579, 'DpC': 1693, 'DpS': 1801, 'DrD': 1907, 'DtC': 2011, 'DuG': 2113, 'EScC': 2221}


{'AdG': 3,
 'AeG': 107,
 'AlD': 211,
 'AuG': 313,
 'CCpG': 419,
 'CbG': 521,
 'CcSE': 631,
 'CdG': 733,
 'CiCP': 839,
 'CpD': 941,
 'CrC': 1049,
 'CtD': 1151,
 'CuG': 1259,
 'DaG': 1361,
 'DdG': 1471,
 'DlA': 1579,
 'DpC': 1693,
 'DpS': 1801,
 'DrD': 1907,
 'DtC': 2011,
 'DuG': 2113,
 'EScC': 2221,
 'FMpG': 2333,
 'G>rG': 2437,
 'GaD': 2539,
 'GbC': 2647,
 'GcG': 2749,
 'GdA': 2851,
 'GdC': 2953,
 'GdD': 3061,
 'GeA': 3163,
 'GiG': 3271,
 'GpBP': 3373,
 'GpCC': 3491,
 'GpMF': 3593,
 'GpPW': 3697,
 'Gr>G': 3803,
 'GuA': 3907,
 'GuC': 4013,
 'GuD': 4127,
 'PBpG': 4229,
 'PCiC': 4337,
 'SpD': 4441,
 'WPpG': 4547}

# Create the original PAM

In [ ]:

time_s = time.time()

val_rels_dict_product = defaultdict(lambda:1)
for i, row in df_train.iterrows():
    val_rels_dict_product[(node2id[row['head']], node2id[row['tail']])] *= rel2id[str(row['rel'])]
    
row = []
col = []
val_rels_prod = []
for key, val in val_rels_dict_product.items():
    row.append(key[0])
    col.append(key[1])
    val_rels_prod.append(val)
    

    
row = np.array(row)
col = np.array(col)
val_rels_prod = np.array(val_rels_prod)

print('Will create the sparse matrices')


A_big = csr_matrix((val_rels_prod, (row, col)), shape=(len(unique_nodes), len(unique_nodes)))


print('Created the sparse matrices')
sparsity = get_sparsity(A_big)
print(A_big.shape, f"Sparsity: {sparsity:.2f} %")

time_prev = time.time()
time_needed = time_prev - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')

# Create the 2-hop PAM

In [76]:
max_order = 2


power_A = [A_big]

time_s = time.time()
time_prev = time_s
for ii in range(1, max_order):
    updated_power = (power_A[-1] * A_big)
    updated_power.sort_indices()
    updated_power.eliminate_zeros()
    power_A.append(updated_power)
    print(f"Sparsity {ii + 1}-hop: {100 * (1 - updated_power.nnz/(updated_power.shape[0]**2)):.2f} %")
    time_prev = time.time()
    time_needed = time_prev - time_s
    print(f'{ii + 1}: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')
time_prev = time.time()
time_needed = time_prev - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')
len(power_A)

Sparsity 2-hop: 77.54 %
2: 25.84444 secs (0.43 mins)
Total time: 25.84455 secs (0.43 mins)


2

## Transform this to a sparse 3d tensor (for easiness in extracting the rules)

In [77]:
def to_sparse_3d(tensorlist):
    for r_i, sparse_ in tqdm.tqdm(enumerate(tensorlist)):
        rows_i, cols_i = sparse_.nonzero()
        vals_i = np.squeeze(np.asarray(sparse_[rows_i, cols_i]))
        if r_i == 0:
            rows = rows_i
            cols = cols_i
            rs = r_i*np.ones((len(rows_i),))
            vals = vals_i
        else:
            rows = np.hstack((rows, rows_i))
            cols = np.hstack((cols, cols_i))
            rs = np.hstack((rs, r_i*np.ones((len(rows_i),))))
            vals = np.hstack((vals, vals_i))
        print(f"Sparsity {r_i+1}-hop: {100 * (1 - sparse_.nnz/(sparse_.shape[0]**2)):.2f} %")
    r,c = sparse_.shape
    
    coo_tensor = sparse.COO(np.array([rows,cols,rs.astype(int)]), vals, shape=(r,c, len(tensorlist)))
    return coo_tensor
time_prev = time.time()
A_tensor = to_sparse_3d(power_A[:4])
time_needed = time_prev - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')

1it [00:00,  6.25it/s]

Sparsity 1-hop: 99.79 %


2it [00:23, 11.66s/it]

Sparsity 2-hop: 77.54 %


Total time: 25.86248 secs (0.43 mins)


# Extract possible metapaths

Now we are going to extract metapaths (or also called rules, because the imply the treats relation) that express the ``*Compound treats Disease*" relation.

1. We first find all pairs of $(i,j)$ for which $P[i,j] = 1151$, where $1151$ is the prime that ``*Compound treats Disease*" is mapped to.
2. Then we keep track of all the different values for these $(i,j)$,that $P^2[i,j] = v \neq 0$.
3. We aggregate all of these metapaths in a DataFrame, keeping also frequencies and other characteristics of the metapath (e.g. support of the rule).

In [103]:
all_rules = []

max_hop = len(power_A)
laplace_smoothing_alpha = 0

time_s = time.time()

# Focus on the pair of nodes that are directly connected with a relation
# In order to find rules/metapaths that indicate such connections
# Only interested in relations of type 1151 which is CtD == Compound treats Disease
rows, cols = (A_big==1151).nonzero()
head_rels = np.array(A_big[rows,cols]).flatten()

for cur_hop in range(1,max_hop):
    cur_power = power_A[cur_hop]
    cur_rules = []
    for index_nonzero, body_rel in enumerate(np.array(cur_power[rows, cols]).flatten()):
        cur_prime_factors = get_primefactors(head_rels[index_nonzero])
        # Same cell rules
        if body_rel != 0:
            for head_rel in cur_prime_factors:
                # Only interested in real relations as head of a rule
                if head_rel in [1151]:
                    cur_rules.append(
                        {
                            'head_rel':head_rel, 
                            "body":body_rel, 
                            "hop":cur_hop + 1, 
                            'type':'same_cell'
                        }
                    )
                
            
    df_rules = pd.DataFrame(cur_rules)
    grouped_by_head_and_body = df_rules.groupby(["head_rel", 'type'])['body'].value_counts().to_dict()
    head_support = df_rules['head_rel'].value_counts().to_dict()
    # This is only using support from the cases we have the rule appear
    #body_support = df_rules.value_counts('body').to_dict()
    # This is for all occurrences of the body
    body_support = pd.Series(cur_power.data).value_counts().to_dict()
    for key in grouped_by_head_and_body:
        head_value, type_, body_value  = key
        all_rules.append({
            "head_rel": head_value, 
            'body':body_value, 
            'head_body_count':grouped_by_head_and_body[key], 
            'body_count':body_support[body_value],
            'head_count':head_support[head_value],
            'hop':cur_hop + 1,
            'type':type_})
    print(f'{cur_hop + 1}-hop level rules: {len(grouped_by_head_and_body)}')
print(f'Total # rules: {len(all_rules)}')

time_prev = time.time()
time_needed = time_prev - time_s
print(f'Total time: {time_needed:.5f} secs ({time_needed/60:.2f} mins)')


all_rules_df = pd.DataFrame(all_rules)
all_rules_df['conf'] = all_rules_df['head_body_count'] / (all_rules_df['body_count'] + laplace_smoothing_alpha)
all_rules_df['head_coverage'] = all_rules_df['head_body_count'] / (all_rules_df['head_count'])
all_rules_df['lift'] = all_rules_df['head_body_count'] / (all_rules_df['head_count'] * all_rules_df['body_count'])
all_rules_df['conviction'] = (1 - all_rules_df['head_count']) / (1 - all_rules_df['conf'] + 0.00001)

print(f'Conf >= 0.01 rules: {len(all_rules_df[all_rules_df["conf"] >=0.01])}')
all_rules_df['body_head_str'] = all_rules_df['body'].astype(str) + "_" + all_rules_df['head_rel'].astype(str)
all_rules_df.sort_values(['head_body_count'], ascending=False)

2-hop level rules: 479
Total # rules: 479
Total time: 132.46191 secs (2.21 mins)
Conf >= 0.01 rules: 459


,head_rel,body,head_body_count,body_count,head_count,hop,type,conf,head_coverage,lift,conviction,body_head_str
0,1151,2194957,23,1284,693,2,same_cell,0.017913,0.033189,0.000026,-7.046146e+02,2194957_1151
1,1151,1207399,20,711,693,2,same_cell,0.028129,0.028860,0.000041,-7.120216e+02,1207399_1151
2,1151,1322819,18,11478,693,2,same_cell,0.001568,0.025974,0.000002,-6.930800e+02,1322819_1151
3,1151,2645638,17,2795,693,2,same_cell,0.006082,0.024531,0.000009,-6.962277e+02,2645638_1151
4,1151,4389914,11,242,693,2,same_cell,0.045455,0.015873,0.000066,-7.249448e+02,4389914_1151
...,...,...,...,...,...,...,...,...,...,...,...,...
200,1151,13356270,1,1,693,2,same_cell,1.000000,0.001443,0.001443,-6.920000e+07,13356270_1151
199,1151,13286587,1,2,693,2,same_cell,0.500000,0.001443,0.000722,-1.383972e+03,13286587_1151
198,1151,13189358,1,1,693,2,same_cell,1.000000,0.001443,0.001443,-6.920000e+07,13189358_1151
197,1151,13176968,1,1,693,2,same_cell,1.000000,0.001443,0.001443,-6.920000e+07,13176968_1151


# Search for the wanted metapaths.

We will first make sure we find the important metapaths as generated by the HetioNet authors.

We first map these metapaths to the corresponding relational chains in the form of the product of the primes of the relations.



In [1]:
metapaths = """CbGaD,
CdGuD,
CrCtD,
CtDrD,
CuGdD"""
metapaths = metapaths.replace("\n", "").split(",")
meta_mapped = []
step = 3
for metapath in metapaths:
    cur_index = 0
    cur_list = []
    while cur_index < len(metapath) - 1:
        cur_name = metapath[cur_index:cur_index+step]
        cur_list.append(rel2id[cur_name])
        cur_index += step -1
    meta_mapped.append({
        "HetioNet": metapath,
        'Mapped':cur_list,
        'Prod': np.prod(cur_list)
    })
metapath_df = pd.DataFrame(meta_mapped)
metapath_df

,HetioNet,Mapped,Prod
0,CbGaD,"[521, 2539]",1322819
1,CdGuD,"[733, 4127]",3025091
2,CrCtD,"[1049, 1151]",1207399
3,CtDrD,"[1151, 1907]",2194957
4,CuGdD,"[1259, 3061]",3853799


# Augmenting the Metapaths

As explained in the paper, the PAM represenation allows for combinations of relations in the metapaths.

To handle these we create the appropriate mappings/namings for the specific metapath examples.

For example the Metapath *{CuG+CbG}GdD* expresses a relational path of the form:
1. It starts from a compound C.
2. One edge connects this compound through the relation *upregulates* to with a Gene G (*CuG*).
3. Another edge connects this compound through the relation *binds* to with a Gene G (*CbG*).
3. Finally, an edge going out from this Gene of the type "downregulates" connects it to disease D (*GdD*)




In [95]:
metapath_df = metapath_df.append(pd.DataFrame([
    {"HetioNet":"{CbG+CuG}GaD", "Mapped":[1780,2539], 'Prod':4519420},
    {"HetioNet":"{CbG+GdC}GaD", "Mapped":[3474,2539], 'Prod':8820486},
    {"HetioNet":"{CuG+CbG}GdD", "Mapped":[1780,3061], 'Prod':5448580},
    ]))


/tmp/ipykernel_3606630/2063964847.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metapath_df = metapath_df.append(pd.DataFrame([


In [96]:
prod2hetio = dict(zip(metapath_df.Prod, metapath_df.HetioNet))

# Load the node names

In [114]:
df_nodes = pd.read_csv("./data/Hetionet/hetionet-v1.0-nodes.tsv", sep='\t')
df_nodes['node_id'] = df_nodes['id'].map(node2id)
nodeid2name = dict(zip(df_nodes['node_id'], df_nodes['name']))
name2nodeid = dict(zip(df_nodes['name'], df_nodes['node_id']))

# Showcase the example of the paper

In the following we explore the metapath *CrC+Ct/pD* (a compound relates to another compound, which both paliates and treats a Disease).

Two pair of *Compounds*, *Diseases*  that **were not directly linked with a ``treats'' relationship in the KG** but are linked through the metapath are:


1. *Hexoprenaline - CrC+Ct/pD -> asthma* : indicating that maybe Hexoprenaline should be used for asthma treatment
2. *Fluticasone Propionate - CrC+Ct/pD -> hematologic cancer: : indicating that maybe Fluticasone Propionate should be used for hematologic cancer treatment

These were validated according to the related bibliography.


In [283]:
rows, cols = (power_A[1]==3401907).nonzero()
for i, row_i in enumerate(rows):
    col_i = cols[i]
    print(f"{nodeid2name[row_i]} - CrC+Ct/pD -> {nodeid2name[col_i]}")
    try:
        gt_rel = " - " + id2rel[power_A[0][row_i, col_i]] + "->"
    except KeyError:
        gt_rel = " not connected "
    print(f"Golden truth: {nodeid2name[row_i]} {gt_rel} {nodeid2name[col_i]}")
    print()

Desoximetasone - CrC+Ct/pD -> hematologic cancer
Golden truth: Desoximetasone  not connected  hematologic cancer

Fluticasone Propionate - CrC+Ct/pD -> hematologic cancer
Golden truth: Fluticasone Propionate  not connected  hematologic cancer

Orciprenaline - CrC+Ct/pD -> asthma
Golden truth: Orciprenaline  - CtD-> asthma

Methyldopa - CrC+Ct/pD -> asthma
Golden truth: Methyldopa  not connected  asthma

Exemestane - CrC+Ct/pD -> hematologic cancer
Golden truth: Exemestane  not connected  hematologic cancer

Dexamethasone - CrC+Ct/pD -> hematologic cancer
Golden truth: Dexamethasone  - CtD-> hematologic cancer

Droxidopa - CrC+Ct/pD -> asthma
Golden truth: Droxidopa  not connected  asthma

Hexoprenaline - CrC+Ct/pD -> asthma
Golden truth: Hexoprenaline  not connected  asthma



# All Metapaths

Finally, here we present all that were found due to the PAMs extended language bias.

We translate them to the corresponding metapaths on the KG.

To extract them we need an ILP solver to decompose the aggregated product of primes to the corresponding base values.

In [227]:
from utils import ILP_solver
metapaths_found = []
for i, body_val in enumerate(all_rules_df['body']):
    found_ = False
    for target_score in metapath_df['Prod']:
        if body_val % target_score == 0:
            print(f'{i} {body_val} := {body_val/target_score} * {target_score} => {prod2hetio[target_score]}')
            metapaths_found.append(prod2hetio[target_score])
            found_ = True
    if not(found_):
        for num_path in range(1,50):
            solved = ILP_solver(metapath_df['Prod'].values.tolist(), body_val, num_path)
            if solved:
                str_map = ""
                for key, iter_ in solved.items():
                    if iter_ > 0:
                        str_map += f"{iter_} * {prod2hetio[int(key)]} + "
                str_map = str_map[:-2]
                print(f'{i} {body_val} := {str_map} => {prod2hetio[target_score]}')
                metapaths_found.append(prod2hetio[target_score])
                break
    

0 2194957 := 1.0 * 2194957 => CtDrD
1 1207399 := 1.0 * 1207399 => CrCtD
2 2414798 := 2.0 * 1207399 => CrCtD
3 4389914 := 2.0 * 2194957 => CtDrD
4 3622197 := 3.0 * 1207399 => CrCtD
5 6584871 := 3.0 * 2194957 => CtDrD
6 3402356 := 1.0 * CrCtD + 1.0 * CtDrD  => {CuG+CbG}GdD
7 4829596 := 4.0 * 1207399 => CrCtD
8 5597313 := 1.0 * CrCtD + 2.0 * CtDrD  => {CuG+CbG}GdD
9 4609755 := 2.0 * CrCtD + 1.0 * CtDrD  => {CuG+CbG}GdD
10 6036995 := 5.0 * 1207399 => CrCtD
11 7244394 := 6.0 * 1207399 => CrCtD
12 7792270 := 1.0 * CrCtD + 3.0 * CtDrD  => {CuG+CbG}GdD
13 8779828 := 4.0 * 2194957 => CtDrD
16 8451793 := 7.0 * 1207399 => CrCtD
17 11634308 := 6.0 * CrCtD + 2.0 * CtDrD  => {CuG+CbG}GdD
19 5817154 := 3.0 * CrCtD + 1.0 * CtDrD  => {CuG+CbG}GdD
20 7024553 := 4.0 * CrCtD + 1.0 * CtDrD  => {CuG+CbG}GdD
21 8231952 := 5.0 * CrCtD + 1.0 * CtDrD  => {CuG+CbG}GdD
22 8999669 := 2.0 * CrCtD + 3.0 * CtDrD  => {CuG+CbG}GdD
23 9219510 := 4.0 * CrCtD + 2.0 * CtDrD  => {CuG+CbG}GdD
24 10426909 := 5.0 * CrCtD + 2.0

For example, the first two paths are *CtDrD* and *CrCtD*, which are mapped to the products $2194957$ and $1207399$ correspondingly.

The third (aggregated) path found is of value $2414798$ which is $1207399\times2$, which essentially expresses 2 different $1207399$ paths, that is 2 distinct *CrCtD* paths.

And so on and so forth.